# Dependency Parsing

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Inline plotting
%matplotlib inline

Dependency parsing is the task of analyzing the structure of sentences in (human) languages.

Why study the structure of sentences? Because we want to be able to interpret language correctly. Humans communicate complex ideas by composing words together into bigger units to convey complex meanings. We need to know what is connected to what.

In question answering tasks, we decompose a question into its syntactical parts like subject and objects in order to figure out what the user is asking.

There are two key tools to solve this problem:
- Constituency Grammar
- Dependency Structure/Grammar (very popular today)

## History

Dependency Structure/Grammar:
- The idea of dependency grammar goes back to Panini's grammar 5th century BCE. 
- It is the basic approach of first millennium Arabic grammarians.
- Modern dependency work often sourced to L. Tesnière (1959)
  - Was dominant approach in the East in 20th Century (Russia, China, ...). 
  - Dependency grammars work well for describing "free-er" word order languages. English is fixed word order language. 
- David Hays, one of the founders of U.S. computational linguistics, built early (first?) dependency parser (Hays 1962)

Constituency/Context-Free Grammar:
- New invention
- Invented in the 20th century by R.S. Wells in 1947. Then by Chomsky.

## Constituency Grammar

Constituency grammar is a way to describe the syntax of a language or the structure of sentences. The idea is to use phrase structure grammers (or context-free grammers) to organize words into nested constituents. A grammer is a set of rules that allows us to generate valid sentences of a language.

<img src="figures/chomsky.png" width="400" />





## Dependency Structure

The other tool, Dependency Structure, describes which words depend on which other words. When one word $w_1$ is dependent on another word $w_1$, then $w_1$ modifies or is an argument of $w_2$.

Suppose we have a sentence:

"Look for the large barking dog by the door in a crate."

- The word *barking* is dependent on *dog* because *barking* modifies *dog*. 
- The word *large* is dependent on *dog* since it modifies *dog*.


Normally, we indicate dependencies by arrows.

<img src="figures/dependency-structure-example.png" width="500" />





### Ambiguous Sentences

An ambiguous sentence can be thought about in terms of different dependencies:

<img src="figures/dependency-structure-ambiguity.png" width="500" />









### Universal Dependencies

Universal Dependencies treebank is a database of sentences where people have manually annotated sentences into dependency graphs. 

<img src="figures/ud-treebank-example.png" width="700" />








Source: [SETS treebank](http://bionlp-www.utu.fi/dep_search/) search maintained by the University of Turku

Building a treebank seems a lot slower and less useful than building a grammar that can generate infinite number of sentences.

A treebank has many advantages:
- Reusability of the labor (grammars cannot be reused because everyone makes up their own)
  - Many parsers, part-of-speech taggers, etc. can be built on it
  - They are valuable resource for linguistics because they give examples of how a language is spoken and syntactic analysis of sentences
- Broad coverage, not just a few intuitions
- Frequencies and distributional information
  - E.g. co-occurence of words
- More crucially, it provides a way to evaluate systems because it provides ground-truth gold standard data



## Dependency Syntax

The theory of dependency syntax postulates that syntactic structure consists of relations between lexical items (i.e., words), normally binary asymmetric relations (i.e., we draw arrows) called dependencies.

The arrows/dependencies are commonly typed with the name of grammatical relations such as subject, prepositional object, apposition, etc. This is referred to as **typed dependency grammar**.

<img src="figures/dependency-structure-example-2.png" width="400" />
















The arrow connects a **head** (or governor/superior/regent) with a **dependent** (or modifier/inferior/subordinate). But there is no consensus of how to draw the arrow. Tesnière had the arrow point from head to dependent.

Usually, dependencies form a tree which means that they have certain graph theorical properties:
- they have a single head (root node). 
- connected
- they are acyclic


Usually, we add a psuodo word ROOT at the start of a sentence so every word is a dependent of precisely one other node. This makes the formalism (math) easy.

## Dependency Conditioning Preferences

What information can help up decide which words are dependent on which other words. We can gather information from various sources:
1. Bilexical affinitiees
   - Discussion of issues is plausible
2. Dependency distance
   - Although there may be some long distance between depency, most of the time there is dependency with nearby words
3. Intervening material
   - Dependencies rarely span intervening verbs or punctuation
4. Valency of heads
   - How many dependents on which side are usual for a head?
   - How likely is a head to have dependents in what number and what size?
   - For example, a word like "the" is not likely to have any dependent at all anywhere.
   - Verbs tend to have many dependents

## Projectivity

Projectivity is defined as "There are no crossing dependency arcs when the words are laid out in their linear order, with all arcs above the words"

Dependencies parallel to a CFG tree must be projective. Forming dependencies by taking 1 child of each category as head.

The figure belows illustrates a **non-projective** dependency because two of arcs are crossing.

<img src="figures/non-projectivity.png" width="500" />




Dependency theory normally allows non-projective structures to account for displaced constituents. You can't easily get the semantics of certain constructions right without these nonprojective dependencies

## Methods of Dependency Parsing

There are many method for dependency parsing:


1. Dynamic programming: Eisner (1996) gives a clever algorithm with complexity $\mathcal{O}(n^3)$, by producing parse items with heads at the ends rather than in the middle. This is normally used for context-free grammars.

2. Graph algorithms: You create a Minimum Spanning Tree (MST) for a sentence. McDonald et al.'s (2005) MSTParser scores dependencies independently using an ML classifier. They use MIRA, for online learning, but it can be something else.

3. Constraint Satisfaction: We can view dependecy parsing as constraint satisfaction problem. Edges that don't satisfy hard constraints are eliminated. Karlsson (1990), etc.

4. **Transition-based parsing** called "deterministic dependency parsing" when it was first introduced. The idea is to greedily choose attachments guided by good machine learning classifiers. MaltParser (Nivre et al. 2008). Has proven highly effective. This method is the dominant way of performing dependency parsing.

## Transition-based Parsing

A transition scheme describes the rules for parsing. It provides a set of legal actions that the parser can take to parse a sentence. One such transition scheme is the arc-standard scheme:

<img src="figures/transition-scheme.png" width="400" />





The parser works like a shift-reduce parser.

The dependency parser performs a sequence of bottom-up actions, roughly like "shift" or "reduce" in a shift-reduce parser. The "reduce" actions are specialized to create dependencies with head on the left or on the right.

The parser has:
- a stack $\sigma$ 
- a buffer $\beta$
- a set of dependency arcs $A$ where attachment decisions are recorded

It can perform three operations:
- Shift
- Left arc
- Right arc


**Shift**: take the word from the top of the buffer and put it on the top of the stack

<img src="figures/shift-operation-example.png" width="200" />









**Left arc:** make attachment decision by adding a word as a dependent to the left



The word "I" is dependent on "ate". Therefore, the left-arc operation removes "I" from the stack and records the attachment decision:

```
A += nsubj(ate -> I)
```

<img src="figures/left-arc-operation-example.png" width="400" />






**Right arc:** make attachment decision by adding a word as a dependent to the right

The word on top of the stack is made dependent on the second-top word of the stack. 

For example, we remove the word "ate" from the stack and record that "fish" is depedent on "ate" i.e., we add an arc/arrow in our dependency tree.

Similarly, "ate" is dependent on "[root]".

<img src="figures/right-arc-operation-example.png" width="400" />










We finish the parsing when we have zero elements in the stack and one element in the buffer.

The remaining question is how do we choose which operation/action to perform next at any point in time?


Nivre and Hall (2005) proposed the following in the MaltParser:

## MaltParser

Each action is predicted by a discriminative classifier (e.g. SVM or softmax classifier) over each legal move. This is possible because we have treebanks of sentences. For each tree structure, there is a unique sequence of shifts, left-arcs and right-arcs that give us the right structure. Given a tree structure, the classifier tell us which operation/action to perform next. There are maximum three choices if the operations are untyped i.e., if we don't classify the grammatically meaning of the word. With typed, we have $|R| \times 2 + 1$ choices.


In the simplest form, there is NO search. But you can profitably do a beam search if you wish (slower but better).

The model's accuracy is fractionally below the state of the art in dependency parsing, but it provides very fast linear time parsing.

## Conventional Feature Representation

As input for the classifier, we can use a combination of features including:
- the word on the top of the stack
- the part-of-speech of that word
- the first word in the buffer
- the part-of-speech of the word in the buffer 

We can represent a configuration of stack, buffer and part-of-speech of words as a large binary vector with approx. 10-100 mio. dimensions.

<img src="figures/conventional-feature-representation.png" width="400" />









We can also represent features that are conjunctions of attributes called indicator features:

<img src="figures/indicator-features.png" width="400" />








- Problem #1: sparse
- Problem #2: incomplete because many configurations will never have been seen by the classifier
- Problem #3: expensive computation. It turns out the symbolic dependency parsers are slow because more than 95% of parsing time is consumed by feature computation and looking up weights.

A better alternative is to learn a dense and compact feature representation (approx. 1000 dimensions) using a dependency parser based on neural network.

## A Neural Dependency Parser

Tokens are extracted based on the stack/buffer positions just like the MaltParser. 

Each word is represented as a $d$-dimensional dense vector (i.e., word embedding). Similar words are expected to have close vectors.

Meanwhile, part-of-speech tags (POS) and dependency labels are also represented as $d$-dimensional vectors. The smaller discrete sets also exhibit many semantical similarities.
- NNS (plural noun) should be close to NN (singular noun)
- NUM (numerical modifier) should be close to AMOD (adjective modifier)

The dense representation is found by looking the tokens up in the embedding matrix and concatenating them into one long vector.

<img src="figures/dense-representation.png" width="400" />











The network architecture:



<img src="figures/network-architecture.png" width="600" />












<img src="figures/further-work.png" width="600" />









## Neural Graph-based Parsers

MSTParser and TurboParser are graph-based parsers.

- Compute a score for every possible dependency for each edge
- Then add an edge from each word to its highest-scoring candidate head
- And repeat the same process for each other word



<img src="figures/neural-graph-based-parser.png" width="600" />









## Evaluation of Dependency Parsing

Evaluating the result of a dependency parser is straightforward:
- Perform the parsing
- Compare it with the ground-truth from the treebank
- The accuracy is the number of dependencies that are correctly parsed divided by the total number of dependencies.
- Unlabelled Accuracy Score (UAS) is measure where we look at the arrows and ignore the labels.
- Labelled Accuracy Score (LAB) 

<img src="figures/evaluation-dependency-parsing.png" width="500" />






